#Задание
* Возьмите англо-русскую пару фраз (https://www.manythings.org/anki/)
* Обучите на них seq2seq по аналогии с занятием. Оцените полученное качество
* Попробуйте добавить +1 рекуррентный слой в encoder и decoder
* Попробуйте заменить GRU ячейки на lstm-ячейки

In [ ]:
%matplotlib inline
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!wget https://www.manythings.org/anki/rus-eng.zip
!unzip rus-eng.zip

--2023-01-21 13:45:10--  https://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15011848 (14M) [application/zip]
Saving to: ‘rus-eng.zip.1’

rus-eng.zip.1       100%[===================>]  14.32M  21.1MB/s    in 0.7s    

2023-01-21 13:45:11 (21.1 MB/s) - ‘rus-eng.zip.1’ saved [15011848/15011848]

Archive:  rus-eng.zip
replace rus.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: _about.txt              


In [ ]:
!tail rus.txt

We need to uphold laws against discrimination — in hiring, and in housing, and in education, and in the criminal justice system. That is what our Constitution and our highest ideals require.	Нам нужно отстаивать законы против дискриминации при найме на работу, в жилищной сфере, в сфере образования и правоохранительной системе. Этого требуют наша Конституция и высшие идеалы.	CC-BY 2.0 (France) Attribution: tatoeba.org #5762728 (BHO) & #6390439 (odexed)
I've heard that you should never date anyone who is less than half your age plus seven. Tom is now 30 years old and Mary is 17. How many years will Tom need to wait until he can start dating Mary?	Я слышал, что никогда не следует встречаться с кем-то вдвое младше вас плюс семь лет. Тому 30 лет, a Мэри 17. Сколько лет Тому нужно ждать до тех пор, пока он сможет начать встречаться с Мэри?	CC-BY 2.0 (France) Attribution: tatoeba.org #10068197 (CK) & #10644473 (notenoughsun)
I do have one final ask of you as your president, the same thing I a

In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+а-яА-Я", r" ", s)
    return s

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('rus.txt', encoding='utf-8').read().strip().split('\n')
    
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    for p in pairs:
        p.pop()

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)

    else:

        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    
    return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
   
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 451436 sentence pairs
Trimmed to 4196 sentence pairs
Counting words...
Counted words:
rus 3874
eng 2060
['если честно, я на тебя полагаюсь .', 'i am relying on you to be honest .']


The Encoder
-----------

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

3m 7s (- 43m 43s) (5000 6%) 3.0874
6m 16s (- 40m 44s) (10000 13%) 2.4859
9m 37s (- 38m 31s) (15000 20%) 2.0094
12m 50s (- 35m 20s) (20000 26%) 1.5932
16m 9s (- 32m 18s) (25000 33%) 1.2226
19m 18s (- 28m 58s) (30000 40%) 0.9548
22m 46s (- 26m 1s) (35000 46%) 0.7210
26m 8s (- 22m 52s) (40000 53%) 0.5365
29m 18s (- 19m 32s) (45000 60%) 0.3982
32m 29s (- 16m 14s) (50000 66%) 0.2902
36m 6s (- 13m 7s) (55000 73%) 0.2155
39m 42s (- 9m 55s) (60000 80%) 0.1447
43m 8s (- 6m 38s) (65000 86%) 0.1152
46m 22s (- 3m 18s) (70000 93%) 0.0846
49m 47s (- 0m 0s) (75000 100%) 0.0624


In [ ]:
evaluateRandomly(encoder1, decoder1, 25)

> она хорошо плавает .
= she swims well .
< she is a good swimmer . <EOS>

> они не будут тебе помогать .
= they aren't going to help you .
< they aren't going to help you . <EOS>

> у него плохое настроение .
= he is in a bad mood .
< he is in a bad mood . <EOS>

> он чрезвычаино чувствителен .
= he is very sensitive .
< he is very sensitive . <EOS>

> ты сильныи .
= you are strong .
< you are strong . <EOS>

> он, наверное, еще жив .
= he is probably still alive .
< he is still alive alive . <EOS>

> мы с томом поддерживаем отношения .
= i am on speaking terms with tom .
< i am on speaking terms with tom . <EOS>

> я пишу письмо .
= i am writing a letter .
< i am writing a letter . <EOS>

> я твоя, а ты моя .
= i am yours and you are mine .
< i am yours and you are mine . <EOS>

> она говорит, что одинока .
= she says that she's lonely .
< she says that she's lonely . <EOS>

> я избранныи .
= i am the chosen one .
< i am the chosen one . <EOS>

> она переплыла реку .
= she swam acros

Добавим в encoder и decoder возможность добавлять рекурретный слой и менять rnn GRU/LSTM

In [ ]:
class EncoderRNN1(nn.Module):
    def __init__(self, input_size, hidden_size, num_rnn = 1, rnnClass = "GRU"):
        super(EncoderRNN1, self).__init__()
        self.hidden_size = hidden_size
        self.num_rnn = num_rnn
        self.embedding = nn.Embedding(input_size, hidden_size)
        
        if rnnClass == "GRU":
          self.rnn = nn.GRU(hidden_size, hidden_size)
        elif rnnClass == "LSTM":
          self.rnn = nn.LSTM(hidden_size, hidden_size)

        if self.num_rnn == 2:
          if rnnClass == "GRU":
              self.rnn_2 = nn.GRU(hidden_size, hidden_size)
          elif rnnClass == "LSTM":
              self.rnn_2 = nn.LSTM(hidden_size, hidden_size)


    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.rnn(output, hidden)
        
        if self.num_rnn == 2:
          output, hidden = self.rnn_2(output, hidden)

        return output, hidden

    def initHidden(self):
      if isinstance(self.rnn, nn.LSTM):
          return (torch.zeros(1, 1, self.hidden_size, device=device), torch.zeros(1, 1, self.hidden_size, device=device))
      return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN1(nn.Module):
    def __init__(self, hidden_size, output_size, num_rnn = 1, rnnClass = "GRU"):
        super(DecoderRNN1, self).__init__()
        self.hidden_size = hidden_size
        self.num_rnn = num_rnn
        self.embedding = nn.Embedding(output_size, hidden_size)

        if rnnClass == "GRU":
          self.rnn = nn.GRU(hidden_size, hidden_size)
        elif rnnClass == "LSTM":
          self.rnn = nn.LSTM(hidden_size, hidden_size)
        if self.num_rnn == 2:
           if rnnClass == "GRU":
               self.rnn_2 = nn.GRU(hidden_size, hidden_size)
           elif rnnClass == "LSTM":
               self.rnn_2 = nn.LSTM(hidden_size, hidden_size)
        
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.rnn(output, hidden)

        if self.num_rnn == 2:
          output, hidden = self.rnn_2(output, hidden)

        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        if isinstance(self.rnn, nn.LSTM):
            return (torch.zeros(1, 1, self.hidden_size, device=device),torch.zeros(1, 1, self.hidden_size, device=device))
        return torch.zeros(1, 1, self.hidden_size, device=device)



#GRU +1 рекуррентный слой в encoder и decoder

In [ ]:
hidden_size = 256
encoder2 = EncoderRNN1(input_lang.n_words, hidden_size, num_rnn=2).to(device)
decoder2 = DecoderRNN1(hidden_size, output_lang.n_words, num_rnn=2).to(device)

trainIters(encoder2, decoder2, 75000, print_every=5000)

4m 58s (- 69m 32s) (5000 6%) 3.3116
9m 26s (- 61m 23s) (10000 13%) 2.8645
13m 59s (- 55m 56s) (15000 20%) 2.6270
18m 33s (- 51m 1s) (20000 26%) 2.4812
23m 15s (- 46m 31s) (25000 33%) 2.2181
27m 51s (- 41m 46s) (30000 40%) 1.9458
32m 30s (- 37m 8s) (35000 46%) 1.6764
37m 6s (- 32m 28s) (40000 53%) 1.4505
41m 42s (- 27m 48s) (45000 60%) 1.2024
46m 24s (- 23m 12s) (50000 66%) 1.0247
51m 4s (- 18m 34s) (55000 73%) 0.8235
55m 44s (- 13m 56s) (60000 80%) 0.6481
60m 31s (- 9m 18s) (65000 86%) 0.5367
65m 41s (- 4m 41s) (70000 93%) 0.4049
70m 30s (- 0m 0s) (75000 100%) 0.3045


In [ ]:
evaluateRandomly(encoder2, decoder2, 25)

> она слишком маленькая, чтобы ходить в школу .
= she is too young to go to school .
< she is too young to go to school . <EOS>

> он всегда на втором плане .
= he is always in the background .
< he is always complaining the others . <EOS>

> я прав .
= i am right .
< i am right . <EOS>

> у него неприятности .
= he is in trouble .
< he is in trouble . <EOS>

> я больна .
= i am sick .
< i am sick . <EOS>

> их здесь нет .
= they aren't here .
< they aren't here . <EOS>

> он боится рака легких .
= he is afraid of getting lung cancer .
< he is afraid of getting lung cancer . <EOS>

> он все еще зависит от своих родителеи .
= he is still dependent on his parents .
< he is still dependent on his parents . <EOS>

> он очень хорошо говорит по-англииски .
= he is a very good speaker of english .
< he is a very good speaker of english . <EOS>

> она шла и пела .
= she sang as she walked .
< she sang as she walked . <EOS>

> вы непоследовательны .
= you aren't consistent .
< you aren't consis


#LSTM

In [ ]:
hidden_size = 256
encoder3 = EncoderRNN1(input_lang.n_words, hidden_size, rnnClass = "LSTM").to(device)
decoder3 = DecoderRNN1(hidden_size, output_lang.n_words, rnnClass = "LSTM").to(device)

trainIters(encoder3, decoder3, 75000, print_every=5000)

3m 30s (- 49m 4s) (5000 6%) 3.1849
6m 51s (- 44m 35s) (10000 13%) 2.6784
10m 15s (- 41m 3s) (15000 20%) 2.3025
13m 36s (- 37m 25s) (20000 26%) 1.9213
17m 3s (- 34m 6s) (25000 33%) 1.5495
20m 27s (- 30m 40s) (30000 40%) 1.2868
23m 49s (- 27m 13s) (35000 46%) 1.0512
27m 18s (- 23m 53s) (40000 53%) 0.8202
30m 44s (- 20m 29s) (45000 60%) 0.6584
34m 6s (- 17m 3s) (50000 66%) 0.5053
37m 44s (- 13m 43s) (55000 73%) 0.3757
41m 28s (- 10m 22s) (60000 80%) 0.2802
45m 7s (- 6m 56s) (65000 86%) 0.2055
48m 41s (- 3m 28s) (70000 93%) 0.1454
52m 24s (- 0m 0s) (75000 100%) 0.1128


In [ ]:
evaluateRandomly(encoder3, decoder3, 25)

> он всегда придирается к другим .
= he is always finding fault with others .
< he is always finding fault with others . <EOS>

> мы подумываем о том, чтобы купить новую мебель .
= we are thinking of buying some new furniture .
< we are thinking of buying some new furniture . <EOS>

> я привык жить один .
= i am accustomed to living alone .
< i am accustomed to living alone . <EOS>

> еи не стоит идти однои .
= she shouldn't go by herself .
< she shouldn't go by herself . <EOS>

> боюсь, он не придет .
= i am afraid he won't come .
< i am afraid he will come . <EOS>

> он в два раза старше ее .
= he is twice as old as she is .
< he is twice as old as she is . <EOS>

> теперь она не одна .
= she isn't lonely now .
< she isn't lonely now . <EOS>

> вечно ты жалуешься на мужа .
= you are always complaining about your husband .
< you are always complaining about your husband . <EOS>

> она с нетерпением ждет своего дня рождения .
= she is looking forward to her birthday party .
< she is lo


#LSTM +1 рекуррентный слой

In [ ]:
hidden_size = 256
encoder4 = EncoderRNN1(input_lang.n_words, hidden_size, num_rnn=2, rnnClass = "LSTM").to(device)
decoder4 = DecoderRNN1(hidden_size, output_lang.n_words, num_rnn=2, rnnClass = "LSTM").to(device)

trainIters(encoder4, decoder4, 75000, print_every=5000)

4m 54s (- 68m 49s) (5000 6%) 3.3758
10m 0s (- 65m 1s) (10000 13%) 3.1475
15m 1s (- 60m 5s) (15000 20%) 3.0216
20m 1s (- 55m 3s) (20000 26%) 2.7356
25m 17s (- 50m 34s) (25000 33%) 2.6157
30m 29s (- 45m 44s) (30000 40%) 2.4327
35m 38s (- 40m 43s) (35000 46%) 2.3440
40m 53s (- 35m 46s) (40000 53%) 2.2050
46m 10s (- 30m 46s) (45000 60%) 2.0994
51m 28s (- 25m 44s) (50000 66%) 1.9524
56m 43s (- 20m 37s) (55000 73%) 1.8216
61m 48s (- 15m 27s) (60000 80%) 1.6574
67m 0s (- 10m 18s) (65000 86%) 1.5555
72m 8s (- 5m 9s) (70000 93%) 1.3570
77m 46s (- 0m 0s) (75000 100%) 1.2079


In [ ]:
evaluateRandomly(encoder4, decoder4, 25)

> ты невыносима .
= you are impossible .
< you are special . <EOS>

> у него хорошие способности к гимнастике .
= he is good at gymnastics .
< he is good at cooking . <EOS>

> он прирожденныи художник .
= he is a born artist .
< he is a simple man . <EOS>

> я думаю поехать за границу .
= i am thinking of going abroad .
< i am thinking of going abroad . <EOS>

> им совсем неинтересно .
= they are not at all interested .
< they are not at all interested . <EOS>

> он водитель автобуса .
= he is a bus driver .
< he is a . . <EOS>

> они разговаривают на кухне .
= they are talking in the kitchen .
< they are in the kitchen . <EOS>

> он всегда дарит подарки своеи жене .
= he is always giving presents to his wife .
< he is always complaining about his his wife . <EOS>

> вы ослеплены любовью .
= you are blinded by love .
< you are taller than me . <EOS>

> она к нему не добра .
= she isn't kind to him .
< she isn't answering her . <EOS>

> я боюсь, что вы потеряетесь .
= i am afraid that y

Итого обучены сети:
* GRU 1 рекуррентный слой   loss = 0.0624    3 неточности на 25 предложений
* GRU 2 рекуррентных слоя   loss = 0.3045    4 / 25 предложений
* LSTM 1 рекуррентный слой  loss = 0.1128    2 / 25 предложений
* LSTM 2 рекуррентных слоя  loss = 1.2079    21 / 25 предложений
Замена GRU на LSTM на время обучения практически не повлияло, а вот добавление рекурретного слоя увеличело время обучения чуть меньше, чем в 2 раза.

Изучение примеров генерации перевода говорит о том, что лучшее качество у LSTM и GRU c 1 рекуррентным слоем.
GRU с 2 рекуррентными слоями тоже неплох. 

А вот LSTM с 2 рекуррентными слоями выглядит местами довольно комично :-)

> ты невыносима .
= you are impossible .
< you are special . <EOS>

> ты абсолютно прав !
= you are absolutely right .
< you are drunk ! <EOS>